<a href="https://colab.research.google.com/github/rma1399/ml-ai-data-science_Learning/blob/main/NLP_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VS = 88584
MAX_LEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VS)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
train_data = sequence.pad_sequences(train_data, maxlen=MAX_LEN)
test_data = sequence.pad_sequences(test_data, maxlen=MAX_LEN)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VS, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, batch_size=BATCH_SIZE, validation_split=0.2)

Epoch 1/10
313/313 [==============================] - 50s 149ms/step - loss: 0.5025 - acc: 0.7424 - val_loss: 0.3320 - val_acc: 0.8654
Epoch 2/10
313/313 [==============================] - 48s 153ms/step - loss: 0.2789 - acc: 0.8924 - val_loss: 0.3316 - val_acc: 0.8778
Epoch 3/10
313/313 [==============================] - 50s 158ms/step - loss: 0.2074 - acc: 0.9244 - val_loss: 0.3339 - val_acc: 0.8682
Epoch 4/10
313/313 [==============================] - 49s 157ms/step - loss: 0.1627 - acc: 0.9438 - val_loss: 0.2981 - val_acc: 0.8876
Epoch 5/10
313/313 [==============================] - 47s 151ms/step - loss: 0.1335 - acc: 0.9546 - val_loss: 0.3684 - val_acc: 0.8528
Epoch 6/10
313/313 [==============================] - 49s 157ms/step - loss: 0.1097 - acc: 0.9635 - val_loss: 0.3601 - val_acc: 0.8678
Epoch 7/10
313/313 [==============================] - 47s 150ms/step - loss: 0.0901 - acc: 0.9705 - val_loss: 0.3772 - val_acc: 0.8852
Epoch 8/10
313/313 [==============================] - 5

In [7]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 23s 29ms/step - loss: 0.4874 - acc: 0.8597
[0.4874115586280823, 0.85971999168396]


In [11]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], maxlen=MAX_LEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [12]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "

  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [17]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1, MAX_LEN))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was really loved it and would watch it again. Was one of the greatest films I have ever seen!"
predict(positive_review)

negative_review = "That movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever seen!"
predict(negative_review)

1/1 [==============================] - 0s 43ms/step
[0.6981587]
1/1 [==============================] - 0s 33ms/step
[0.34662142]
